# Load libraries

In [2]:
# load pandas and numpy
import pandas as pd
import numpy as np
import re
import os
import sys
from typing import Sequence, Dict

sys.path.append(os.path.abspath('./paper_results'))
from parser import avg_results

# Table 1: accuracy

In [4]:
datasets = ['iglu', 'colas', 'dubosson', 'hall', 'weinstock'] # iglu is Broll in the paper, otherwise alphabetical order
models = ['arima', 'linreg', 'xgboost', 'gluformer', 'latentode',  'nhits', 'tft', 'transformer']

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = None
        (dict_errors, _), (dict_errors_std, _) = avg_results(model_names, model_names_with_covs)
    else:
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (dict_errors, _), (dict_errors_std, _) = avg_results(model_names, model_names_with_covs)

In [9]:
datasets = ['iglu', 'colas', 'dubosson', 'hall', 'weinstock'] # iglu is Broll in the paper, otherwise alphabetical order
models = ['arima', 'linreg', 'xgboost', 'gluformer', 'latentode',  'nhits', 'tft', 'transformer']

def color(x):
    return r'\multicolumn{2}{c}{\textcolor{red}{+' + str(round(x, 2)) + '\%}}' if x > 0 else r' \multicolumn{2}{c}{\textcolor{blue}{' + str(round(x, 2)) + '\%}}'

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        print(model)
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = None
        (dict_errors, _), _ = avg_results(model_names, model_names_with_covs)

        diff_errors_no_covs = (dict_errors['ood']['no_covs'] - dict_errors['id']['no_covs']) / dict_errors['id']['no_covs'] * 100
        print(' & '.join([color(x) for x in diff_errors_no_covs.reshape(-1).tolist()]))
    
    else:
        print(model)
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (dict_errors, _), _ = avg_results(model_names, model_names_with_covs)

        diff_errors_no_covs = (dict_errors['ood']['no_covs'] - dict_errors['id']['no_covs']) / dict_errors['id']['no_covs'] * 100
        print(' & '.join([color(x) for x in diff_errors_no_covs.reshape(-1).tolist()]))

arima
\multicolumn{2}{c}{\textcolor{red}{+11.59\%}} & \multicolumn{2}{c}{\textcolor{red}{+12.01\%}} & \multicolumn{2}{c}{\textcolor{red}{+2.02\%}} & \multicolumn{2}{c}{\textcolor{red}{+1.49\%}} & \multicolumn{2}{c}{\textcolor{red}{+38.56\%}} & \multicolumn{2}{c}{\textcolor{red}{+31.81\%}} &  \multicolumn{2}{c}{\textcolor{blue}{-4.79\%}} &  \multicolumn{2}{c}{\textcolor{blue}{-5.08\%}} & \multicolumn{2}{c}{\textcolor{red}{+18.47\%}} & \multicolumn{2}{c}{\textcolor{red}{+18.56\%}}
linreg
\multicolumn{2}{c}{\textcolor{red}{+2.51\%}} & \multicolumn{2}{c}{\textcolor{red}{+1.29\%}} & \multicolumn{2}{c}{\textcolor{red}{+1.27\%}} & \multicolumn{2}{c}{\textcolor{red}{+1.38\%}} & \multicolumn{2}{c}{\textcolor{red}{+30.01\%}} & \multicolumn{2}{c}{\textcolor{red}{+19.41\%}} & \multicolumn{2}{c}{\textcolor{red}{+6.46\%}} & \multicolumn{2}{c}{\textcolor{red}{+4.58\%}} & \multicolumn{2}{c}{\textcolor{red}{+14.5\%}} & \multicolumn{2}{c}{\textcolor{red}{+14.22\%}}
xgboost
 \multicolumn{2}{c}{\textcolor

In [3]:
datasets = ['iglu', 'colas', 'dubosson', 'hall', 'weinstock'] # iglu is Broll in the paper, otherwise alphabetical order
models = ['arima', 'linreg', 'xgboost', 'gluformer', 'latentode',  'nhits', 'tft', 'transformer']

def color(x):
    return r'\textcolor{red}{+' + str(round(x, 2)) + '\%}' if x > 0 else r'\textcolor{blue}{' + str(round(x, 2)) + '\%}'

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        pass   
    else:
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (dict_errors, _), _ = avg_results(model_names, model_names_with_covs)

        print(model)
        diff_errors = (dict_errors['id']['covs'] - dict_errors['id']['no_covs']) / dict_errors['id']['no_covs'] * 100
        diff_errors = diff_errors.tolist()
        print(' & '.join([color(y) for x in diff_errors for y in x]))

linreg
\textcolor{blue}{-14.82\%} & \textcolor{blue}{-13.34\%} & \textcolor{red}{+5.54\%} & \textcolor{red}{+5.75\%} & \textcolor{red}{+2.84\%} & \textcolor{red}{+0.61\%} & \textcolor{red}{+6.17\%} & \textcolor{red}{+5.09\%} & \textcolor{blue}{-1.54\%} & \textcolor{blue}{-1.08\%}
xgboost
\textcolor{red}{+8.53\%} & \textcolor{red}{+3.22\%} & \textcolor{blue}{-0.72\%} & \textcolor{blue}{-0.5\%} & \textcolor{blue}{-1.36\%} & \textcolor{blue}{-2.8\%} & \textcolor{red}{+6.22\%} & \textcolor{red}{+7.11\%} & \textcolor{red}{+0.99\%} & \textcolor{red}{+1.36\%}
nhits
\textcolor{red}{+17.43\%} & \textcolor{red}{+21.29\%} & \textcolor{red}{+53.14\%} & \textcolor{red}{+59.48\%} & \textcolor{red}{+74.38\%} & \textcolor{red}{+89.1\%} & \textcolor{red}{+6.21\%} & \textcolor{red}{+8.05\%} & \textcolor{red}{+0.88\%} & \textcolor{red}{+0.93\%}
tft
\textcolor{red}{+6.86\%} & \textcolor{red}{+12.21\%} & \textcolor{red}{+15.86\%} & \textcolor{red}{+15.93\%} & \textcolor{red}{+0.35\%} & \textcolor{red}{+0.0

In [5]:
datasets = ['iglu', 'colas', 'dubosson', 'hall', 'weinstock'] # iglu is Broll in the paper, otherwise alphabetical order
models = ['arima', 'linreg', 'xgboost', 'gluformer', 'latentode',  'nhits', 'tft', 'transformer']

def color(x):
    return r'\multicolumn{2}{c}{\textcolor{red}{+' + str(round(x, 2)) + '\%}}' if x > 0 else r' \multicolumn{2}{c}{\textcolor{blue}{' + str(round(x, 2)) + '\%}}'

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = None
        (dict_errors, _), _ = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{2}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f}' for x in dict_errors['id']['no_covs'].reshape(-1).tolist()]))

        print(r'\\')

        print('& \crossmark & ')
        print(r'OOD & ' + '&'.join([f'{x:.2f}' for x in dict_errors['ood']['no_covs'].reshape(-1).tolist()]))

        print(r'\\')
        print('\midrule')
        print(r'\rowcolor{lightgray}')

        print(r'\multicolumn{3}{c|}{$\min \Delta$(ID, OD)\%}&')
        diff_errors = np.mean((dict_errors['ood']['no_covs'] - dict_errors['id']['no_covs']) / 
                              dict_errors['id']['no_covs'] * 100, 
                              axis=1)
        print(' & '.join([color(x) for x in diff_errors.reshape(-1).tolist()]))

        print(r'\\')
        print('\midrule')
    
    else:
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (dict_errors, _), _ = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{6}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')

        print(r'ID & ' + '&'.join([f'{x:.2f}' for x in dict_errors['id']['no_covs'].reshape(-1).tolist()]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f}' for x in dict_errors['id']['covs'].reshape(-1).tolist()]))
        
        print(r'\\')
        print('\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}')
        
        print('& \multicolumn{2}{c|}{Improv.} &')  
        diff_errors = np.mean((dict_errors['id']['covs'] - dict_errors['id']['no_covs']) / 
                              dict_errors['id']['no_covs'] * 100,
                                axis=1)
        print(' & '.join([color(x) for x in diff_errors.reshape(-1).tolist()]))
        
        print(r'\\')
        print('\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}')
        
        print('& \crossmark &')
        print(r'OD & ' + '&'.join([f'{x:.2f}' for x in dict_errors['ood']['no_covs'].reshape(-1).tolist()]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'OD & ' + '&'.join([f'{x:.2f}' for x in dict_errors['ood']['covs'].reshape(-1).tolist()]))
        
        print(r'\\')
        print('\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}')
        
        print('& \multicolumn{2}{c|}{Improv.} &')  
        diff_errors = np.mean((dict_errors['ood']['covs'] - dict_errors['ood']['no_covs']) / 
                              dict_errors['ood']['no_covs'] * 100,
                                axis=1)
        print(' & '.join([color(x) for x in diff_errors.reshape(-1).tolist()]))

        print(r'\\')
        print('\midrule')
        print(r'\rowcolor{lightgray}')

        print(r'\multicolumn{3}{c|}{$\min \Delta$(ID, OD)\%}&')
        diff_errors_no_covs = np.mean((dict_errors['ood']['no_covs'] - dict_errors['id']['no_covs']) / 
                                       dict_errors['id']['no_covs'] * 100,
                                       axis=1)
        diff_errors_covs = np.mean((dict_errors['ood']['covs'] - dict_errors['id']['covs']) /
                                    dict_errors['id']['covs'] * 100,
                                    axis=1)
        diff_errors = np.minimum(diff_errors_no_covs, diff_errors_covs)
        print(' & '.join([color(x) for x in diff_errors.reshape(-1).tolist()]))

        print(r'\\')
        print('\midrule')

\multirow{2}{*}{\rotatebox{90}{ARI}} & \crossmark & 
ID & 10.53&8.67&5.80&4.80&13.53&11.06&8.63&7.34&13.40&11.25
\\
& \crossmark & 
OOD & 11.75&9.71&5.91&4.87&18.75&14.58&8.22&6.97&15.87&13.34
\\
\midrule
\rowcolor{lightgray}
\multicolumn{3}{c|}{$\min \Delta$(ID, OD)\%}&
\multicolumn{2}{c}{\textcolor{red}{+11.8\%}} & \multicolumn{2}{c}{\textcolor{red}{+1.75\%}} & \multicolumn{2}{c}{\textcolor{red}{+35.18\%}} &  \multicolumn{2}{c}{\textcolor{blue}{-4.94\%}} & \multicolumn{2}{c}{\textcolor{red}{+18.51\%}}
\\
\midrule
\multirow{6}{*}{\rotatebox{90}{LIN}} & \crossmark & 
ID & 11.68&9.71&5.26&4.35&12.07&9.97&7.38&6.33&13.60&11.46
\\
& \checkmark & 
ID & 9.95&8.41&5.56&4.60&12.41&10.03&7.84&6.66&13.39&11.34
\\
\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}
& \multicolumn{2}{c|}{Improv.} &
 \multicolumn{2}{c}{\textcolor{blue}{-14.08\%}} & \multicolumn{2}{c}{\textcolor{red}{+5.65\%}} & \multicolumn{2}{c}{\textcolor{red}{+1.73\%}} & \multicolu

# Table 2: probabilistic fit

In [5]:
datasets = ['iglu', 'colas', 'dubosson', 'hall', 'weinstock'] # iglu is Broll in the paper, otherwise alphabetical order
models = ['linreg', 'xgboost', 'gluformer', 'latentode',  'nhits', 'tft', 'transformer']

def color_min(x):
    return r'\textcolor{red}{+' + str(round(x, 2)) + '\%}' if x > 0 else r'\textcolor{blue}{' + str(round(x, 2)) + '\%}'
def color_max(x):
    return r'\textcolor{blue}{+' + str(round(x, 2)) + '\%}' if x > 0 else r'\textcolor{red}{' + str(round(x, 2)) + '\%}'

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = None
        (_, dict_errors), _ = avg_results(model_names, model_names_with_covs)

        print(model)
        diff_errors = (dict_errors['ood']['no_covs'] - dict_errors['id']['no_covs']) / np.abs(dict_errors['id']['no_covs']) * 100
        string = [[color_max(x[0]), color_min(x[1])] for x in diff_errors.tolist()]
        string = [item for sublist in string for item in sublist]
        print(' & '.join(string))
    
    else:
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (_, dict_errors), _ = avg_results(model_names, model_names_with_covs)

        print(model)
        diff_errors_no_covs = (dict_errors['ood']['no_covs'] - dict_errors['id']['no_covs']) / np.abs(dict_errors['id']['no_covs']) * 100
        string = [[color_max(x[0]), color_min(x[1])] for x in diff_errors_no_covs.tolist()]
        string = [item for sublist in string for item in sublist]
        print(' & '.join(string))

linreg
\textcolor{red}{-0.65\%} & \textcolor{red}{+24.98\%} & \textcolor{blue}{+0.33\%} & \textcolor{red}{+2.93\%} & \textcolor{red}{-0.02\%} & \textcolor{blue}{-7.62\%} & \textcolor{blue}{+0.33\%} & \textcolor{red}{+4.43\%} & \textcolor{red}{-0.85\%} & \textcolor{blue}{-3.1\%}
xgboost
\textcolor{red}{-0.88\%} & \textcolor{red}{+67.42\%} & \textcolor{blue}{+0.59\%} & \textcolor{blue}{-4.55\%} & \textcolor{blue}{+3.16\%} & \textcolor{red}{+5.02\%} & \textcolor{blue}{+1.16\%} & \textcolor{blue}{-14.37\%} & \textcolor{red}{-0.83\%} & \textcolor{red}{+2.45\%}
gluformer
\textcolor{blue}{+6.72\%} & \textcolor{red}{+106.76\%} & \textcolor{red}{-50.33\%} & \textcolor{blue}{-29.83\%} & \textcolor{blue}{+45.64\%} & \textcolor{red}{+83.63\%} & \textcolor{blue}{+7.69\%} & \textcolor{red}{+9.24\%} & \textcolor{blue}{+3.33\%} & \textcolor{red}{+6.23\%}
latentode
\textcolor{red}{-13.67\%} & \textcolor{red}{+6.5\%} & \textcolor{blue}{+15.89\%} & \textcolor{blue}{-4.01\%} & \textcolor{blue}{+42.14\%} &

/tmp/ipykernel_65873/1381547546.py:27: RuntimeWarning: invalid value encountered in divide
  diff_errors_no_covs = (dict_errors['ood']['no_covs'] - dict_errors['id']['no_covs']) / np.abs(dict_errors['id']['no_covs']) * 100


In [6]:
datasets = ['iglu', 'colas', 'dubosson', 'hall', 'weinstock'] # iglu is Broll in the paper, otherwise alphabetical order
models = ['linreg', 'xgboost', 'gluformer', 'latentode',  'nhits', 'tft', 'transformer']

def color_min(x):
    return r'\textcolor{red}{+' + str(round(x, 2)) + '\%}' if x > 0 else r'\textcolor{blue}{' + str(round(x, 2)) + '\%}'
def color_max(x):
    return r'\textcolor{blue}{+' + str(round(x, 2)) + '\%}' if x > 0 else r'\textcolor{red}{' + str(round(x, 2)) + '\%}'

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        pass
    else:
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (_, dict_errors), _ = avg_results(model_names, model_names_with_covs)

        print(model)
        diff_errors = (dict_errors['id']['covs'] - dict_errors['id']['no_covs']) / np.abs(dict_errors['id']['no_covs']) * 100
        string = [[color_max(x[0]), color_min(x[1])] for x in diff_errors.tolist()]
        string = [item for sublist in string for item in sublist]
        print(' & '.join(string))

linreg
\textcolor{blue}{+0.15\%} & \textcolor{red}{+5.99\%} & \textcolor{blue}{+0.25\%} & \textcolor{red}{+24.47\%} & \textcolor{red}{-0.41\%} & \textcolor{red}{+11.39\%} & \textcolor{red}{-7.67\%} & \textcolor{red}{+97.36\%} & \textcolor{blue}{+0.13\%} & \textcolor{blue}{-1.67\%}
xgboost
\textcolor{red}{-1.22\%} & \textcolor{red}{+0.59\%} & \textcolor{blue}{+0.12\%} & \textcolor{blue}{-7.2\%} & \textcolor{blue}{+0.13\%} & \textcolor{blue}{-6.98\%} & \textcolor{red}{-0.31\%} & \textcolor{blue}{-1.3\%} & \textcolor{red}{-0.15\%} & \textcolor{blue}{-4.62\%}
nhits
\textcolor{red}{-3.63\%} & \textcolor{blue}{-37.79\%} & \textcolor{red}{-1.68\%} & \textcolor{red}{+91.12\%} & \textcolor{red}{-4.19\%} & \textcolor{blue}{-21.68\%} & \textcolor{red}{-0.07\%} & \textcolor{blue}{-24.77\%} & \textcolor{red}{-0.01\%} & \textcolor{blue}{-5.46\%}
tft
\textcolor{red}{nan\%} & \textcolor{red}{+94.6\%} & \textcolor{red}{nan\%} & \textcolor{red}{+114.61\%} & \textcolor{red}{nan\%} & \textcolor{red}{+7.57

/tmp/ipykernel_65873/1688311718.py:18: RuntimeWarning: invalid value encountered in divide
  diff_errors = (dict_errors['id']['covs'] - dict_errors['id']['no_covs']) / np.abs(dict_errors['id']['no_covs']) * 100


In [7]:
datasets = ['iglu', 'colas', 'dubosson', 'hall', 'weinstock'] # iglu is Broll in the paper, otherwise alphabetical order
models = ['linreg', 'xgboost', 'gluformer', 'latentode',  'nhits', 'tft', 'transformer']

def color_min(x):
    return r'\textcolor{red}{+' + str(round(x, 2)) + '\%}' if x > 0 else r'\textcolor{blue}{' + str(round(x, 2)) + '\%}'
def color_max(x):
    return r'\textcolor{blue}{+' + str(round(x, 2)) + '\%}' if x > 0 else r'\textcolor{red}{' + str(round(x, 2)) + '\%}'

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = None
        (_, dict_errors), _ = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{2}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f}' for x in dict_errors['id']['no_covs'].reshape(-1).tolist()]))

        print(r'\\')

        print('& \crossmark & ')
        print(r'OOD & ' + '&'.join([f'{x:.2f}' for x in dict_errors['ood']['no_covs'].reshape(-1).tolist()]))

        print(r'\\')
        print('\midrule')
        print(r'\rowcolor{lightgray}')

        print(r'\multicolumn{3}{c|}{$\min \Delta$(ID, OD)\%}&')
        diff_errors = (dict_errors['ood']['no_covs'] - dict_errors['id']['no_covs']) / np.abs(dict_errors['id']['no_covs']) * 100
        string = [[color_max(x[0]), color_min(x[1])] for x in diff_errors.tolist()]
        string = [item for sublist in string for item in sublist]
        print(' & '.join(string))

        print(r'\\')
        print('\midrule')
    
    else:
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (_, dict_errors), _ = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{6}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')

        print(r'ID & ' + '&'.join([f'{x:.2f}' for x in dict_errors['id']['no_covs'].reshape(-1).tolist()]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f}' for x in dict_errors['id']['covs'].reshape(-1).tolist()]))
        
        print(r'\\')
        print('\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}')
        
        print('& \multicolumn{2}{c|}{Improv.} &')  
        diff_errors = (dict_errors['id']['covs'] - dict_errors['id']['no_covs']) / np.abs(dict_errors['id']['no_covs']) * 100
        string = [[color_max(x[0]), color_min(x[1])] for x in diff_errors.tolist()]
        string = [item for sublist in string for item in sublist]
        print(' & '.join(string))
        
        print(r'\\')
        print('\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}')
        
        print('& \crossmark &')
        print(r'OD & ' + '&'.join([f'{x:.2f}' for x in dict_errors['ood']['no_covs'].reshape(-1).tolist()]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'OD & ' + '&'.join([f'{x:.2f}' for x in dict_errors['ood']['covs'].reshape(-1).tolist()]))
        
        print(r'\\')
        print('\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}')
        
        print('& \multicolumn{2}{c|}{Improv.} &')  
        diff_errors = (dict_errors['ood']['covs'] - dict_errors['ood']['no_covs']) / np.abs(dict_errors['ood']['no_covs']) * 100
        string = [[color_max(x[0]), color_min(x[1])] for x in diff_errors.tolist()]
        string = [item for sublist in string for item in sublist]
        print(' & '.join(string))

        print(r'\\')
        print('\midrule')
        print(r'\rowcolor{lightgray}')

        print(r'\multicolumn{3}{c|}{$\min \Delta$(ID, OD)\%}&')
        diff_errors_no_covs = (dict_errors['ood']['no_covs'] - dict_errors['id']['no_covs']) / np.abs(dict_errors['id']['no_covs']) * 100
        diff_errors_covs = (dict_errors['ood']['covs'] - dict_errors['id']['covs']) / np.abs(dict_errors['id']['covs']) * 100
        diff_errors = np.empty_like(diff_errors_no_covs)
        diff_errors[:, 1] = np.maximum(diff_errors_no_covs[:, 0], diff_errors_covs[:, 0])
        diff_errors[:, 1] = np.minimum(diff_errors_no_covs[:, 1], diff_errors_covs[:, 1])
        string = [[color_max(x[0]), color_min(x[1])] for x in diff_errors.tolist()]
        string = [item for sublist in string for item in sublist]
        print(' & '.join(string))

        print(r'\\')
        print('\midrule')

\multirow{6}{*}{\rotatebox{90}{LIN}} & \crossmark & 
ID & -9.89&0.12&-9.19&0.15&-10.10&0.18&-9.56&0.10&-10.14&0.11
\\
& \checkmark & 
ID & -9.87&0.13&-9.17&0.19&-10.15&0.21&-10.30&0.19&-10.12&0.11
\\
\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}
& \multicolumn{2}{c|}{Improv.} &
\textcolor{blue}{+0.15\%} & \textcolor{red}{+5.99\%} & \textcolor{blue}{+0.25\%} & \textcolor{red}{+24.47\%} & \textcolor{red}{-0.41\%} & \textcolor{red}{+11.39\%} & \textcolor{red}{-7.67\%} & \textcolor{red}{+97.36\%} & \textcolor{blue}{+0.13\%} & \textcolor{blue}{-1.67\%}
\\
\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}
& \crossmark &
OD & -9.95&0.15&-9.16&0.15&-10.11&0.17&-9.53&0.10&-10.22&0.11
\\
& \checkmark & 
OD & -10.24&0.55&-9.16&0.17&-12.08&0.48&-10.42&0.23&-11.13&0.21
\\
\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}
& \multicolumn{2}{c|}{Improv.

/tmp/ipykernel_65873/1608005624.py:54: RuntimeWarning: invalid value encountered in divide
  diff_errors = (dict_errors['id']['covs'] - dict_errors['id']['no_covs']) / np.abs(dict_errors['id']['no_covs']) * 100
/tmp/ipykernel_65873/1608005624.py:74: RuntimeWarning: invalid value encountered in divide
  diff_errors = (dict_errors['ood']['covs'] - dict_errors['ood']['no_covs']) / np.abs(dict_errors['ood']['no_covs']) * 100
/tmp/ipykernel_65873/1608005624.py:84: RuntimeWarning: invalid value encountered in divide
  diff_errors_no_covs = (dict_errors['ood']['no_covs'] - dict_errors['id']['no_covs']) / np.abs(dict_errors['id']['no_covs']) * 100
/tmp/ipykernel_65873/1608005624.py:85: RuntimeWarning: invalid value encountered in divide
  diff_errors_covs = (dict_errors['ood']['covs'] - dict_errors['id']['covs']) / np.abs(dict_errors['id']['covs']) * 100


# Table 1: short version

In [8]:
datasets = ['iglu', 'colas', 'dubosson', 'hall', 'weinstock'] # iglu is Broll in the paper, otherwise alphabetical order
models = ['arima', 'linreg', 'xgboost', 'gluformer', 'latentode',  'nhits', 'tft', 'transformer']

model_errors = []
for model in models:
    model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
    model_names_with_covs = None
    (dict_errors, _), _ = avg_results(model_names, model_names_with_covs)
    model_errors.append(dict_errors['id']['no_covs'])

In [9]:
model_errors = np.mean(np.array(model_errors), axis=-1)
# find best models with lowest error
best_models = np.argsort(model_errors, axis=0)
best_models_set = []
for i, dataset in enumerate(datasets):
    bm_dataset = [models[j] for j in best_models[:, i]]
    print(f'{dataset}: {bm_dataset[0]}')
    best_models_set.append(bm_dataset[0])
print(set(best_models_set))

iglu: arima
colas: linreg
dubosson: linreg
hall: latentode
weinstock: transformer
{'arima', 'transformer', 'latentode', 'linreg'}


# Table 2: short version

In [10]:
datasets = ['iglu', 'colas', 'dubosson', 'hall', 'weinstock'] # iglu is Broll in the paper, otherwise alphabetical order
models = ['arima', 'linreg', 'xgboost', 'gluformer', 'latentode',  'nhits', 'tft', 'transformer']

model_errors = []
for model in models:
    model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
    model_names_with_covs = None
    (_, dict_errors), _ = avg_results(model_names, model_names_with_covs)
    model_errors.append(dict_errors['id']['no_covs'])

In [11]:
model_likelihood = np.array(model_errors)[:,:,0]
model_cal = np.array(model_errors)[:,:,1]
# find best models with highest likelihood
model_likelihood[np.isnan(model_likelihood)] = -np.inf
model_likelihood[model_likelihood == 0] = -np.inf # models that have exactly 0 likelihood do not support likelihood
best_models = np.argsort(model_likelihood, axis=0)
best_models_set = []
for i, dataset in enumerate(datasets):
    bm_dataset = [models[j] for j in best_models[:, i]]
    print(f'{dataset}: {bm_dataset[-1]}')
    best_models_set.append(bm_dataset[-1])
print(set(best_models_set))

# find best models with lowest cal error
model_cal[np.isnan(model_cal)] = np.inf
best_models = np.argsort(model_cal, axis=0)
best_models_set = []
for i, dataset in enumerate(datasets):
    bm_dataset = [models[j] for j in best_models[:, i]]
    print(f'{dataset}: {bm_dataset[0]}')
    best_models_set.append(bm_dataset[0])
print(set(best_models_set))



iglu: gluformer
colas: gluformer
dubosson: gluformer
hall: gluformer
weinstock: gluformer
{'gluformer'}
iglu: gluformer
colas: tft
dubosson: gluformer
hall: gluformer
weinstock: tft
{'gluformer', 'tft'}


# Clearing the results

In [12]:
# import os

# # Set the directory path to the folder containing the output files
# folder_path = './output'

# # Loop through each file in the folder
# for filename in os.listdir(folder_path):
#     file_path = os.path.join(folder_path, filename)
#     # Open the file in read mode if the file starts with transformer
#     if filename.startswith('transformer') or \
#             filename.startswith('tft') or \
#                 filename.startswith('linreg') or \
#                     filename.startswith('xgboost') or \
#                         filename.startswith('nhits'):
#         with open(file_path, 'r') as f:
#             lines = f.readlines()
            
#         # Loop through the lines in reverse order
#         for i in range(len(lines)-1, -1, -1):
#             if lines[i].startswith('Best value: '):
#                 # Delete all lines after the line starting with "Best value: "
#                 del lines[i+1:]
#                 break
        
#         # Open the file in write mode and write the modified lines back to the file
#         with open(file_path, 'w') as f:
#             f.writelines(lines)


# Tables 3-6 (std. error in metric estimates)

In [42]:
models = ['tft', 'transformer', 'xgboost', 'nhits']
# 'gluformer', 'latentode'
datasets = ['iglu', 'hall', 'colas', 'weinstock', 'dubosson']
metrics = ['ID mean of (MSE, MAE)', 'OOD mean of (MSE, MAE)',
           'ID median of (MSE, MAE)', 'OOD median of (MSE, MAE)',
           'ID likelihoods', 'OOD likelihoods', 
           'ID calibration errors', 'OOD calibration errors']

for model in models:
    for dataset in datasets:
        # open model file
        with open(f'../output/{model}_covariates_{dataset}.txt', 'rb') as f:
            lines = f.readlines()
        results = {}
        for metric in metrics:
            results[metric] = {}
            for seed in [1, 2]:
                results[metric][seed] = []
                for model_seed in range(10, 20):
                    # find lines starting with Model Seed: model_seed Seed: seed metric: 
                    prev_line_has_calibration = False
                    for line in lines:
                        line = line.decode()
                        if f'Model Seed: {model_seed} Seed: {seed} {metric}' in str(line):
                            vals = re.findall(r'-?\d+\.\d+(?:e-\d+)?', line)
                            results[metric][seed].append([float(x) for x in vals])
                            if 'calibration' in metric:
                                prev_line_has_calibration = True
                        # check that line has no text just numbers and symbols
                        check = re.findall(r'[a-zA-Z]', line)
                        if prev_line_has_calibration and len(check) == 0:
                            prev_line_has_calibration = False
                            vals = re.findall(r'-?\d+\.\d+(?:e-\d+)?', line)
                            results[metric][seed][-1].extend([float(x) for x in vals])
        for metric in metrics:
            for seed in [1, 2]:
                results[metric][seed] = np.array(results[metric][seed])
                results[metric][seed] = np.mean(results[metric][seed], axis=0)
            results[metric] = np.std([results[metric][1], results[metric][2]], axis=0)
        
        new_lines = []
        for line in lines:
            line = line.decode()
            for metric in metrics:
                if ('Seed' not in line) and (metric in line):
                    if len(results[metric]) > 0:
                        line = line.strip('\n') + f' +- {results[metric]} \n' if len(results[metric]) > 1 else line.strip('\n') + f' +- {results[metric][0]} \n'
            new_lines.append(line)
        # write new lines to file
        with open(f'../output/{model}_covariates_{dataset}.txt', 'wb') as f:
            for line in new_lines:
                f.write(line.encode())


In [7]:
# RMSE table

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = None
        (dict_errors, _), (dict_errors_std, _) = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{2}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['no_covs'][:, 0].tolist(), \
          dict_errors_std['id']['no_covs'][:, 0].tolist())]))

        print(r'\\')

        print('& \crossmark & ')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['no_covs'][:, 0].tolist(), \
          dict_errors_std['ood']['no_covs'][:, 0].tolist())]))

        print(r'\\')
        print('\midrule')
    
    else:
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (dict_errors, _), (dict_errors_std, _) = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{4}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')

        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['no_covs'][:, 0].tolist(), \
          dict_errors_std['id']['no_covs'][:, 0].tolist())]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['covs'][:, 0].tolist(), \
          dict_errors_std['id']['covs'][:, 0].tolist())]))
        
        print(r'\\')
        print('\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}')
        
        print('& \crossmark &')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['no_covs'][:, 0].tolist(), \
          dict_errors_std['ood']['no_covs'][:, 0].tolist())]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['covs'][:, 0].tolist(), \
          dict_errors_std['ood']['covs'][:, 0].tolist())]))

        print(r'\\')
        print('\midrule')

\multirow{2}{*}{\rotatebox{90}{ARI}} & \crossmark & 
ID & 10.53 +- 4.69&5.80 +- 0.83&13.53 +- 6.37&8.63 +- 1.50&13.40 +- 1.77
\\
& \crossmark & 
OD & 11.75 +- 7.26&5.91 +- 2.38&18.75 +- 15.10&8.22 +- 5.02&15.87 +- 4.31
\\
\midrule
\multirow{4}{*}{\rotatebox{90}{LIN}} & \crossmark & 
ID & 11.68 +- 3.61&5.26 +- 0.57&12.07 +- 5.49&7.38 +- 1.29&13.60 +- 1.59
\\
& \checkmark & 
ID & 9.95 +- 2.73&5.56 +- 0.84&12.41 +- 5.71&7.84 +- 1.85&13.39 +- 1.60
\\
\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}
& \crossmark &
OD & 11.98 +- 7.08&5.33 +- 1.60&15.69 +- 12.17&7.86 +- 4.23&15.58 +- 4.64
\\
& \checkmark & 
OD & 23.30 +- 24.85&5.54 +- 1.60&203114.47 +- 287247.24&14.22 +- 17.87&15.66 +- 4.48
\\
\midrule
\multirow{4}{*}{\rotatebox{90}{XGB}} & \crossmark & 
ID & 12.80 +- 2.62&6.42 +- 1.04&21.18 +- 5.38&7.58 +- 1.23&13.63 +- 1.67
\\
& \checkmark & 
ID & 13.89 +- 3.04&6.37 +- 1.75&20.89 +- 5.62&8.05 +- 1.41&13.77 +- 1.67
\\
\cmidrule(lr){4-5} \cmid

In [9]:
# MAE table

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = None
        (dict_errors, _), (dict_errors_std, _) = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{2}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['no_covs'][:, 1].tolist(), \
          dict_errors_std['id']['no_covs'][:, 1].tolist())]))

        print(r'\\')

        print('& \crossmark & ')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['no_covs'][:, 1].tolist(), \
          dict_errors_std['ood']['no_covs'][:, 1].tolist())]))

        print(r'\\')
        print('\midrule')
    
    else:
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (dict_errors, _), (dict_errors_std, _) = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{4}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')

        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['no_covs'][:, 1].tolist(), \
          dict_errors_std['id']['no_covs'][:, 1].tolist())]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['covs'][:, 1].tolist(), \
          dict_errors_std['id']['covs'][:, 1].tolist())]))
        
        print(r'\\')
        print('\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}')
        
        print('& \crossmark &')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['no_covs'][:, 1].tolist(), \
          dict_errors_std['ood']['no_covs'][:, 1].tolist())]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['covs'][:, 1].tolist(), \
          dict_errors_std['ood']['covs'][:, 1].tolist())]))

        print(r'\\')
        print('\midrule')

\multirow{2}{*}{\rotatebox{90}{ARI}} & \crossmark & 
ID & 8.67 +- 0.74&4.80 +- 0.04&11.06 +- 0.98&7.34 +- 0.16&11.25 +- 0.10
\\
& \crossmark & 
OD & 9.71 +- 1.93&4.87 +- 0.38&14.58 +- 4.75&6.97 +- 1.19&13.34 +- 0.52
\\
\midrule
\multirow{4}{*}{\rotatebox{90}{LIN}} & \crossmark & 
ID & 9.71 +- 0.37&4.35 +- 0.03&9.97 +- 1.00&6.33 +- 0.09&11.46 +- 0.11
\\
& \checkmark & 
ID & 8.41 +- 0.24&4.60 +- 0.04&10.03 +- 1.11&6.66 +- 0.18&11.34 +- 0.10
\\
\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}
& \crossmark &
OD & 9.83 +- 1.58&4.41 +- 0.20&11.90 +- 3.51&6.62 +- 0.91&13.09 +- 0.61
\\
& \checkmark & 
OD & 16.80 +- 9.45&4.57 +- 0.19&67548.59 +- 135072.57&10.02 +- 6.68&13.16 +- 0.57
\\
\midrule
\multirow{4}{*}{\rotatebox{90}{XGB}} & \crossmark & 
ID & 11.50 +- 0.31&5.49 +- 0.08&19.09 +- 0.32&6.55 +- 0.09&11.61 +- 0.08
\\
& \checkmark & 
ID & 11.87 +- 0.24&5.46 +- 0.21&18.55 +- 0.82&7.02 +- 0.12&11.77 +- 0.16
\\
\cmidrule(lr){4-5} \cmidrule(lr){6

In [10]:
# likelihood table

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = None
        (_, dict_errors), (_, dict_errors_std) = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{2}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['no_covs'][:, 0].tolist(), \
          dict_errors_std['id']['no_covs'][:, 0].tolist())]))

        print(r'\\')

        print('& \crossmark & ')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['no_covs'][:, 0].tolist(), \
          dict_errors_std['ood']['no_covs'][:, 0].tolist())]))

        print(r'\\')
        print('\midrule')
    
    else:
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (_, dict_errors), (_, dict_errors_std) = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{4}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')

        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['no_covs'][:, 0].tolist(), \
          dict_errors_std['id']['no_covs'][:, 0].tolist())]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['covs'][:, 0].tolist(), \
          dict_errors_std['id']['covs'][:, 0].tolist())]))
        
        print(r'\\')
        print('\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}')
        
        print('& \crossmark &')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['no_covs'][:, 0].tolist(), \
          dict_errors_std['ood']['no_covs'][:, 0].tolist())]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['covs'][:, 0].tolist(), \
          dict_errors_std['ood']['covs'][:, 0].tolist())]))

        print(r'\\')
        print('\midrule')

\multirow{2}{*}{\rotatebox{90}{ARI}} & \crossmark & 
ID & nan +- nan&nan +- nan&nan +- nan&nan +- nan&nan +- nan
\\
& \crossmark & 
OD & nan +- nan&nan +- nan&nan +- nan&nan +- nan&nan +- nan
\\
\midrule
\multirow{4}{*}{\rotatebox{90}{LIN}} & \crossmark & 
ID & -9.89 +- 0.01&-9.19 +- 0.01&-10.10 +- 0.16&-9.56 +- 0.03&-10.14 +- 0.00
\\
& \checkmark & 
ID & -9.87 +- 0.03&-9.17 +- 0.01&-10.15 +- 0.17&-10.30 +- 1.47&-10.12 +- 0.00
\\
\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}
& \crossmark &
OD & -9.95 +- 0.14&-9.16 +- 0.06&-10.11 +- 0.28&-9.53 +- 0.17&-10.22 +- 0.03
\\
& \checkmark & 
OD & -10.24 +- 0.30&-9.16 +- 0.06&-12.08 +- 3.94&-10.42 +- 1.49&-11.13 +- 1.83
\\
\midrule
\multirow{4}{*}{\rotatebox{90}{XGB}} & \crossmark & 
ID & -9.94 +- 0.02&-9.42 +- 0.01&-10.55 +- 0.02&-9.68 +- 0.01&-10.20 +- 0.00
\\
& \checkmark & 
ID & -10.06 +- 0.06&-9.40 +- 0.02&-10.54 +- 0.01&-9.70 +- 0.00&-10.21 +- 0.00
\\
\cmidrule(lr){4-5} \cmidrule(lr){6-

In [11]:
# calibration table

for model in models:
    if model in ['arima', 'gluformer', 'latentode']: # no covariates
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = None
        (_, dict_errors), (_, dict_errors_std) = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{2}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['no_covs'][:, 1].tolist(), \
          dict_errors_std['id']['no_covs'][:, 1].tolist())]))

        print(r'\\')

        print('& \crossmark & ')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['no_covs'][:, 1].tolist(), \
          dict_errors_std['ood']['no_covs'][:, 1].tolist())]))

        print(r'\\')
        print('\midrule')
    
    else:
        model_names = [f'../output/{model}_{dataset}.txt' for dataset in datasets]
        model_names_with_covs = [f'../output/{model}_covariates_{dataset}.txt' for dataset in datasets]
        (_, dict_errors), (_, dict_errors_std) = avg_results(model_names, model_names_with_covs)

        print(r'\multirow{4}{*}{\rotatebox{90}{'+ model[:3].upper() + r'}} & \crossmark & ')

        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['no_covs'][:, 1].tolist(), \
          dict_errors_std['id']['no_covs'][:, 1].tolist())]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'ID & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['id']['covs'][:, 1].tolist(), \
          dict_errors_std['id']['covs'][:, 1].tolist())]))
        
        print(r'\\')
        print('\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}')
        
        print('& \crossmark &')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['no_covs'][:, 1].tolist(), \
          dict_errors_std['ood']['no_covs'][:, 1].tolist())]))

        print(r'\\')
        
        print('& \checkmark & ')
        print(r'OD & ' + '&'.join([f'{x:.2f} +- {y:.2f}' for (x, y) in zip(dict_errors['ood']['covs'][:, 1].tolist(), \
          dict_errors_std['ood']['covs'][:, 1].tolist())]))

        print(r'\\')
        print('\midrule')

\multirow{2}{*}{\rotatebox{90}{ARI}} & \crossmark & 
ID & nan +- nan&nan +- nan&nan +- nan&nan +- nan&nan +- nan
\\
& \crossmark & 
OD & nan +- nan&nan +- nan&nan +- nan&nan +- nan&nan +- nan
\\
\midrule
\multirow{4}{*}{\rotatebox{90}{LIN}} & \crossmark & 
ID & 0.12 +- 0.01&0.15 +- 0.00&0.18 +- 0.02&0.10 +- 0.00&0.11 +- 0.00
\\
& \checkmark & 
ID & 0.13 +- 0.02&0.19 +- 0.01&0.21 +- 0.02&0.19 +- 0.20&0.11 +- 0.00
\\
\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}
& \crossmark &
OD & 0.15 +- 0.04&0.15 +- 0.01&0.17 +- 0.02&0.10 +- 0.02&0.11 +- 0.00
\\
& \checkmark & 
OD & 0.55 +- 0.39&0.17 +- 0.02&0.48 +- 0.58&0.23 +- 0.18&0.21 +- 0.20
\\
\midrule
\multirow{4}{*}{\rotatebox{90}{XGB}} & \crossmark & 
ID & 0.07 +- 0.01&0.10 +- 0.01&0.07 +- 0.01&0.09 +- 0.00&0.11 +- 0.00
\\
& \checkmark & 
ID & 0.07 +- 0.01&0.09 +- 0.01&0.06 +- 0.01&0.09 +- 0.00&0.10 +- 0.00
\\
\cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cm